#### references / libraries

https://docs.python.org/3/library/colorsys.html convert rgb to hsl, hsv and back

https://pillow.readthedocs.io/en/stable/reference/Image.html

https://pillow.readthedocs.io/en/stable/reference/ImageColor.html

https://webcolors.readthedocs.io/en/latest/install.html

https://adamspannbauer.github.io/2018/03/02/app-icon-dominant-colors/ (get_dominant_color())


__note:__

if saving html from a plot, 
to remove the resize popups on ipyplot tables, regex searches below, replace with ''

`<a href=.*\n.*\n.*</a>`

`<a href=.*\n.*<span class="ipyplot-img-expand"/>\n.*</a>`

In [1]:
# this turns off future deprecation warnings from ipyplot, change the commenting to undo and re-run
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='default', category=FutureWarning)

import math
import colorsys

from PIL import Image as Img, ImageColor
import numpy as np
import cv2
import ipyplot
import webcolors

from colour import Color
from color_utils import get_dominant_color

# these are two ordered lists of block names & block textures as PIL Imgs
from ruined import concretes_names, concretes, stones_names, stones, etces, etces_names, roads_names, roads

all_block_names = concretes_names + etces_names + stones_names + roads_names
all_block_imgs = concretes + etces + stones +roads

In [7]:
def dom_rgb(pil_img: Img, clusters=None, img_size=None, as_list=None, return_comp=None):
    """
    returns tuple(r,g,b) of the dominant color as  of a PIL Img
    the image as a list using color_utils.dominant_color
    pil_img = Img.open('path/to/file.png')
    img_drgb(pil_img)
    [56.2423442, 34.0834233, 70.1234123]
    """
    if not clusters:
        clusters = 3
    if not img_size:
        img_size = (16, 16)
    
    # convert PIL Img to HSV np.array for opencv to call get_dominant_color
    bgr_image = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)

    # 
    dc = get_dominant_color(hsv_image, clusters,img_size)
    # create a square showing dominant color of equal size to input image
    dom_color_hsv = np.full(hsv_image.shape, dc, dtype='uint8')
    # convert to bgr color space for display
    dom_color_bgr = cv2.cvtColor(dom_color_hsv, cv2.COLOR_HSV2BGR)
    
    # retun_comp returns a side-by-side comparison img of the original texture vs the dominant color
    if return_comp:
        
        # combine the original image and the dominant color img
        output_image = np.hstack((bgr_image, dom_color_bgr))

        # convert back to RGB from BGR and return PIL Img
        out_im = cv2.cvtColor(output_image,  cv2.COLOR_BGR2RGB)
        return Img.fromarray(out_im, mode="RGB")
    else:
        # convert dom color image to RGB and create a new Pil Img
        out_im = Img.fromarray(cv2.cvtColor(dom_color_bgr,  cv2.COLOR_BGR2RGB))
        # load array of pixels
        px = out_im.load()
        # get the rgb of pixel 0,0
        ci_rgb = px[0, 0]
        return list(ci_rgb) if as_list else ci_rgb


In [9]:
def rgb_color_dist(rgb1: tuple, rgb2: tuple):
    r1,g1, b1 = rgb1
    r2,g2, b2 = rgb2
    
    diff = math.sqrt((r2 - r1)**2 +  (g2 - g1)**2 +  (b2 - b1)**2) 
    return diff



col1 = dom_rgb(concretes[1])
col2 = dom_rgb(concretes[2])

r1,g1, b1 = col1
r2,g2, b2 = col2
channel_sub_sq = (r2 - r1)**2 +  (g2 - g1)**2 +  (b2 - b1)**2
dist = math.sqrt(channel_sub_sq )


print(dist, rgb_color_dist(dom_rgb(concretes[1]), dom_rgb(concretes[2])))

166.27687752661222 166.27687752661222


In [10]:
def rgb_to_hex(rgb, with_hash=None):
    
    der_hex = '%02x%02x%02x' % rgb
    if with_hash:
        der_hex =  f"#{der_hex}"
    return der_hex

def img_hex(pil_img, with_hash=None):
    rgbl = dom_rgb(pil_img)
    da_hex = rgb_to_hex(tuple(dom_rgb(pil_img)))
    if with_hash:
        da_hex =  f"#{da_hex}"
    return da_hex
    
def img_compliment_rgb(pil_img, as_list=None):
    rgbl = dom_rgb(pil_img, as_list=True)
    compliment = [255 - rgbl[0], 255 - rgbl[1], 255 - rgbl[2]]
    return compliment if as_list else tuple(compliment)

rgb_to_hex((108, 80, 134), with_hash=True)


'#6c5086'

In [11]:
def get_color_ramp(base_hex:str, steps:int):
    c = Color(base_hex)
    cl = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=0.70)
    cd = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=0.10)
    
    return [color.get_web() for color in cd.range_to(cl, 2)]

black, white = get_color_ramp('#6c5086', 2)

black

'#1a1320'

In [12]:
# concrete_dict = { n: {f"{n}.png" : i }  for n, i in dict(zip(concretes_names, concretes)).items()}



info_dict_blocks, img_dict_blocks = ({},{})

for n, i in dict(zip(all_block_names, all_block_imgs)).items():
    domn_rgb = dom_rgb(i)
    compl_rgb = img_compliment_rgb(i)
    dom_hex = rgb_to_hex(domn_rgb)
    c_hex = rgb_to_hex(compl_rgb)
    
    compl_block = {'dist': 1000.0, 'name' : 'bad_distance'}
    
    #print(f"\n====distances from {n}{domn_rgb}'s complimentary rgb {compl_rgb}")
    for n2, i2 in dict(zip(all_block_names, all_block_imgs)).items():
        this_dist = rgb_color_dist(compl_rgb, dom_rgb(i2))
        #print(f"\t{this_dist} -- {n2}")
        if this_dist < compl_block['dist']:
            compl_block = {'dist': this_dist, 'name' : n2}
    
    img_dict_blocks[n] = {
        'texture': i,
        'color_chip' : Img.new('RGB', (16, 16), domn_rgb),
        'comp_img': dom_rgb(i, return_comp=True)
    }
            
    info_dict_blocks[n] = {
        "hex": dom_hex,
        "rgb": domn_rgb,
        "compl_rgb": compl_rgb,
        "compl_block": compl_block["name"],
        "compl_dist": compl_block["dist"]
    }
    



In [13]:
#print(len(info_dict_blocks), len(concretes_names + etces_names + stones_names))

ordered_block_info_keys = list(info_dict_blocks.keys())
ordered_block_info_keys.sort()
ordered_block_info = {i: info_dict_blocks[i] for i in ordered_block_info_keys}
 
print(ordered_block_info)

{'andesite': {'hex': '595d54', 'rgb': (89, 93, 84), 'compl_rgb': (166, 162, 171), 'compl_block': 'beige_tile', 'compl_dist': 20.639767440550294}, 'asphalt': {'hex': '515151', 'rgb': (81, 81, 81), 'compl_rgb': (174, 174, 174), 'compl_block': 'beige_tile', 'compl_dist': 23.430749027719962}, 'asphalt_darker': {'hex': '404040', 'rgb': (64, 64, 64), 'compl_rgb': (191, 191, 191), 'compl_block': 'white_concrete', 'compl_dist': 5.196152422706632}, 'beige_tile': {'hex': 'ada698', 'rgb': (173, 166, 152), 'compl_rgb': (82, 89, 103), 'compl_block': 'plastic_blue', 'compl_dist': 9.433981132056603}, 'beige_tile_2': {'hex': 'ada698', 'rgb': (173, 166, 152), 'compl_rgb': (82, 89, 103), 'compl_block': 'plastic_blue', 'compl_dist': 9.433981132056603}, 'black_concrete': {'hex': '434040', 'rgb': (67, 64, 64), 'compl_rgb': (188, 191, 191), 'compl_block': 'white_concrete', 'compl_dist': 4.242640687119285}, 'blackstone': {'hex': '34343c', 'rgb': (52, 52, 60), 'compl_rgb': (203, 203, 195), 'compl_block': 'whi

In [14]:
concretes_names


['brown_concrete',
 'gray_concrete',
 'white_concrete',
 'light_gray_concrete',
 'black_concrete']

In [28]:
c_comps, c_names  = ([],[])


for nm, c in dict(zip(concretes_names + etces_names + stones_names  , concretes + etces + stones )).items():

    ci = dom_rgb(c, return_comp=True)
    px = ci.load()
    ci_rgb = px[17, 0]
    c_comps.append(ci)
    c_names.append(f"{nm}<br>rgb{ci_rgb}<br>{rgb_to_hex(ci_rgb)}")

ipyplot.plot_class_representations(c_comps, c_names)

In [31]:
ipyplot.plot_images(stones, stones_names)

In [11]:
s_comps, s_names  = ([],[])


for snm, s in dict(zip(stones_names, stones )).items():

    si = dom_rgb(s, return_comp=True)
    px = si.load()
    si_rgb = px[17, 0]
    s_comps.append(si)
    s_names.append(f"{snm}<br>rgb{si_rgb}<br>{rgb_to_hex(si_rgb)}")

ipyplot.plot_images(s_comps, s_names)

In [12]:
e_comps, e_names  = ([],[])

for enm, e in dict(zip(etces_names, etces)).items():

    ei = dom_rgb(e, return_comp=True)
    epx = ei.load()
    ei_rgb = epx[17, 0]
    e_comps.append(ei)
    e_names.append(f"{enm}<br>rgb{ei_rgb}<br>{rgb_to_hex(ei_rgb)}")

ipyplot.plot_images(e_comps, e_names)

In [13]:
ipyplot.plot_images(etces, etces_names)


In [14]:
## test color conversion there and back again

im = Img.open('blocks_to_ruin/concrete/cyan_concrete.png')

# convert rgb to bgr and then to hsv
bgr_im = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
hsv_im = cv2.cvtColor(bgr_im, cv2.COLOR_BGR2HSV)

# get the dom color
dc = get_dominant_color(hsv_im, 3)

# create a comp img between im and a monocolor img from the value of dc
dom_color_hsv = np.full(hsv_im.shape, dc, dtype='uint8')
# convert monocolor img back to bgr
dom_color_bgr = cv2.cvtColor(dom_color_hsv, cv2.COLOR_HSV2BGR)
out_im = cv2.cvtColor(dom_color_bgr,  cv2.COLOR_BGR2RGB)
# print(dc, dom_color_hsv[0] )

# back to RGB
bgr_im2 = cv2.cvtColor(hsv_im, cv2.COLOR_HSV2BGR)
rgb_im2 = cv2.cvtColor(bgr_im, cv2.COLOR_BGR2RGB)
im2 = Img.fromarray(rgb_im2, mode="RGB")

#print(rgb_im2[0], dc)
print('cyan_concrete dominant rgb color')
print(dom_rgb(im, 3), dom_rgb(im, 3, as_list=True ))

ipyplot.plot_images([
    im, im2, dom_rgb(im, return_comp=True)
], ['original', 'converted', 'dom_rgb() comp'])


cyan_concrete dominant rgb color
(75, 122, 130) [75, 122, 130]


In [17]:
# for name, info in info_dict_blocks.items():
#     this_compl_rgb = info["compl_rgb"]
    
#     best_distance = {'dist': 1000.0, 'name' : 'bad_distance'}
#     for onm, oinf  in info_dict_blocks.items():
#         other_rgb = oinf['rgb']
#         this_dist = rgb_color_dist(this_compl_rgb, other_rgb)
#         if this_dist < best_distance['dist']:
#             best_distance = {'dist': this_dist, 'name' : onm}
            
#     print(f"{name} {best_distance['name']}'")
#     info_dict_blocks[name]['comp_name'] = best_distance['name']
#     info_dict_blocks[name]['comp_dist'] = best_distance['dist']
#     #print(name, info['rgb'], info["compl_rgb"])

In [20]:
ipyplot.plot_class_representations(concretes + stones + etces, concretes_names + stones_names + etces_names)